# Chapter 3: Cross-validation

The definition of the author follows: cross-validation is a step in the process of building a machine learning model which helps us ensure that our models fit the data accurately and also ensures that we do not overfit.